### Can we do compositional ReFT?

I have:

- **A ReFT for continuing sentences in German**
- **A ReFT for following instructions**

Can I just combine them and have an "instruction-following model that speaks German"? Let's see!

First of all, you need to know the notations of **subspace**, **linear subspace**, and **orthonormal linear subspaces**! You can read more about these in Atticus's [causal abstraction paper](https://arxiv.org/abs/2301.04709). Briefly, here is what they are:

- **subspace**: you can think of it as a single dimension of an NN's representation in the NN's original basis (learned one).
- **linear subspace**: representation in a changed basis, and the new basis is a linear combination (i.e., any rotation) of the original basis.
- **orthonormal linear subspaces**: if the new linear subspace is produced by an orthonormal projection, then each dimension (or sub-subspace, sorry about the confusion here) in that new basis is orthogonal to each other. Or more strictly speaking, *it maintains the orthogonality if the original basis has it*.

So for ReFT, we can theoretically leverage the notation of subspace, and train different subspaces for different tasks separately, and snap them together at the inference time! Let's see if it will work in practice.

In [1]:
import sys

sys.path.append('/home/jovyan/pyreft')

In [2]:
import torch
import transformers
from datasets import load_dataset, concatenate_datasets

from pyreft import (
    TaskType,
    get_reft_model,
    ReftConfig,
    ReftTrainerForCausalLM, 
    ReftDataCollator,
    ReftSupervisedDataset,
    LoreftIntervention
)

prompt_no_input_template = """Below is an instruction that \
describes a task. Write a response that appropriately \
completes the request.

### Instruction:
%s

### Response:
"""

device = "cuda" if torch.cuda.is_available() else "cpu"


##################################################
# Subspace partitions:

# Let's have a LoReFT of rank 8, and assign
# - the first 4 rank to german sentence completion
# - the next 4 rank to instruction following
##################################################
HELLASWAG_SUBSPACE = [0,1,2,3]
INSTRUCT_SUBSPACE = [4,5,6,7]

def preprocess_hellaswag_de_for_reft(examples):
    label = int(examples["label"])
    if len(examples["endings_de"]) < 4:
        output = examples["endings_de"][-1]
    else:
        output = examples["endings_de"][label]
    examples["instruction"] = examples["ctx"]
    examples["output"] = output
    examples["subspaces"] = HELLASWAG_SUBSPACE
    return examples

def preprocess_ultrafeedback_for_reft(examples):
    examples["subspaces"] = INSTRUCT_SUBSPACE
    return examples

raw_dataset = load_dataset("LeoLM/HellaSwag_de")
drop_features = list(raw_dataset["train"].features.keys())
raw_dataset = raw_dataset.map(preprocess_hellaswag_de_for_reft)
hellaswag_de_dataset = raw_dataset.remove_columns(drop_features)["train"]

raw_dataset = load_dataset("json", data_files="./ultrafeedback_1k.json")["train"]
raw_dataset = raw_dataset.map(preprocess_ultrafeedback_for_reft)
ultrafeedback_dataset = raw_dataset.remove_columns(["input"])

subspace_dataset = concatenate_datasets([hellaswag_de_dataset, ultrafeedback_dataset])

class SubloreftIntervention(LoreftIntervention):
    """
    This is a LoReFT that supports subspace interventions!
    """
    def forward(
        self, base, source=None, subspaces=None
    ):
        assert subspaces is not None
        output = []
        
        rotated_base = self.rotate_layer(base)
        diff = self.act_fn(self.learned_source(base)) - rotated_base
        
        batched_subspace = []
        batched_weights = []
        
        for example_i in range(len(subspaces)):
            LHS = diff[example_i, 0, subspaces[example_i]].unsqueeze(dim=0)
            RHS = self.rotate_layer.weight[..., subspaces[example_i]].T
            batched_subspace += [LHS]
            batched_weights += [RHS]

        batched_subspace = torch.stack(batched_subspace, dim=0)
        batched_weights = torch.stack(batched_weights, dim=0)
        output = base + torch.bmm(batched_subspace, batched_weights)

        return self.dropout(output.to(base.dtype))

### Loading the base LM (LLaMA-1 here! not Llama-2)

In [3]:
# load model (take 1 min)
model_name_or_path = "yahma/llama-7b-hf" # yahma/llama-7b-hf or yahma/llama-13b-hf
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name_or_path, 
    torch_dtype=torch.bfloat16, 
    device_map=device,
    cache_dir='/home/jovyan/.cache/huggingface/hub')

# get tokenizer
model_max_length = 512
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name_or_path, model_max_length=model_max_length,
    padding_side="right", use_fast=False)
tokenizer.pad_token = tokenizer.unk_token

config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


### Load rank 8 LoReFT config

In [4]:
TARGET_LAYER = 15

# get reft model
reft_config = ReftConfig(representations={
    "layer": TARGET_LAYER, "component": "block_output",
    "intervention": SubloreftIntervention(
    embed_dim=model.config.hidden_size, low_rank_dimension=8)})
reft_model = get_reft_model(model, reft_config)
reft_model.print_trainable_parameters()

trainable intervention params: 65,544 || trainable model params: 0
model params: 6,738,415,616 || trainable%: 0.0009726915603776257


### Load dataset

Note that in total, we only have **2,000 training examples**, since LoReFT works with low resource settings - a bonus we did not fully explore in the paper.

In [6]:
train_dataset = ReftSupervisedDataset(
    "Subloreft", None, tokenizer, dataset=subspace_dataset,
    **{"num_interventions": 1, "position": "l1", "share_weights": False},
    input_field="input", instruction_field="instruction", output_field="output",
)
data_collator_fn = transformers.DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    padding="longest"
)
data_collator = ReftDataCollator(data_collator=data_collator_fn)

  0%|          | 0/2000 [00:00<?, ?it/s]


KeyError: 'subspaces'

### Training!

Note that we are not training a shared subspace for two tasks! We are training them individually by providing the `subspaces` field in the input! Checkout [pyvene](https://github.com/stanfordnlp/pyvene) about how to use `subspaces` field - there are other stuff we haven't tried.

In [41]:
# train
training_args = transformers.TrainingArguments(
    num_train_epochs=3.0, output_dir="./tmp", learning_rate=5e-3, report_to=[],
    per_device_train_batch_size=8, logging_steps=50
)
trainer = ReftTrainerForCausalLM(
    model=reft_model, tokenizer=tokenizer, args=training_args, 
    train_dataset=train_dataset, eval_dataset=None, data_collator=data_collator)
trainer.train()

Step,Training Loss
50,1.247600
100,1.278800
150,1.202400
200,1.230300
250,1.230800
300,1.204000
350,1.269600
400,1.188200
450,1.165800
500,1.243600


Checkpoint destination directory ./tmp/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.


Directory './tmp/checkpoint-500/intervenable_model' already exists.


TrainOutput(global_step=750, training_loss=1.2146810963948567, metrics={'train_runtime': 439.3119, 'train_samples_per_second': 13.658, 'train_steps_per_second': 1.707, 'total_flos': 0.0, 'train_loss': 1.2146810963948567, 'epoch': 3.0})

### Interact with the German sentence completion subspace

In [71]:
instruction = "How to kill a linux process?"

prompt = prompt_no_input_template % instruction
prompt = tokenizer(prompt, return_tensors="pt").to(device)

base_unit_location = prompt["input_ids"].shape[-1] - 1  # last position
_, reft_response = reft_model.generate(
    prompt, unit_locations={"sources->base": (None, [[[base_unit_location]]])},
    subspaces=[[HELLASWAG_SUBSPACE]],
    intervene_on_prompt=True, max_new_tokens=128, do_sample=False, 
    no_repeat_ngram_size=5, repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id, early_stopping=True
)
print(tokenizer.decode(reft_response[0], skip_special_tokens=True))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
How to kill a linux process?

### Response:
Es wird ein Linux-Prozess getötet, indem man ihn mit dem Befehl "kill" tötet.


### Interact with the instruction following subspace

In [73]:
_, reft_response = reft_model.generate(
    prompt, unit_locations={"sources->base": (None, [[[base_unit_location]]])},
    subspaces=[[INSTRUCT_SUBSPACE]],
    intervene_on_prompt=True, max_new_tokens=512, do_sample=False, 
    no_repeat_ngram_size=5, repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id, early_stopping=True
)
print(tokenizer.decode(reft_response[0], skip_special_tokens=True))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
How to kill a linux process?

### Response:
To kill a Linux process, you can use the `kill` command with the PID (process ID) of the process you want to terminate. For example, if you want to kill the process with PID 123456789, you would run the following command:

```
$ kill -9 123
```

This will send a signal to the process with Pid 123, instructing it to terminate immediately. The `-9` flag indicates that the process should be terminated forcefully and without any further warning or prompts.

Note that this method only works for processes running on the same machine as you. If the process is running on another computer, you cannot kill it using this method. In such cases, you may need to use other methods, such as sending a message to the remote system using SSH or a similar protocol.


### Interact with both subspaces, partially!

To interact with both of them, you can simply change the `subspaces` field at the inference time to any combinations you want!

In [74]:
base_unit_location = prompt["input_ids"].shape[-1] - 1  # last position
_, reft_response = reft_model.generate(
    prompt, unit_locations={"sources->base": (None, [[[base_unit_location]]])},
    # unfortunately including 4,5 will break this, please explore why! maybe too strong?
    # the output length dist of these two datasets are so different, so ...
    subspaces=[[[0,1,2,3,6,7]]], 
    intervene_on_prompt=True, max_new_tokens=512, do_sample=False, 
    no_repeat_ngram_size=5, repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id, early_stopping=True
)
print(tokenizer.decode(reft_response[0], skip_special_tokens=True))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
How to kill a linux process?

### Response:
Es gibt verschiedene Möglichkeiten, um einen Linux-Prozess zu löschen.

1. Mit dem Kommando "kill" kann man den Prozess beenden.
2. Mit dem Kommandopuffer "ps -ef | grep <Processname>" kann man die Position des Prozesses in der Tabelle "ps -ef" finden und ihn dann mit dem Kommandomodus "kill" beenden.


This is an early sneak-peek of our **Feature Compartmentalization**, and **Schematic ReFT**! Stay tuned, and explore with us! We think there are basically infinite number of causal pathways in the neural network waiting for us to explore!